# Diffusion-Shock Inpainting in $SE(2)$
Diffusion-shock inpainting (DS) is a technique to fill in missing structures in images, developed in ["Diffusion-Shock Inpainting" (2023) by K. Schaefer and J. Weickert](https://link.springer.com/chapter/10.1007/978-3-031-31975-4_45) and the follow-up paper ["Regularised Diffusion-Shock Inpainting" (2023) by K. Schaefer and J. Weickert](https://arxiv.org/abs/2309.08761). In this notebook, we will look at DS applied to images lifted into $SE(2)$.

In $\mathbb{R}^2$, we can describe DS in a PDE-based formulation as
$$
\partial_t u = g(\lvert \nabla (G_{\nu} * u) \rvert^2) \underbrace{\Delta u}_{\textrm{Diffusion}} - \left(1 - g(\lvert \nabla (G_{\nu} * u) \rvert^2)\right) \underbrace{\mathrm{sgn}(\partial_{\vec{w} \vec{w}} (G_{\sigma} * u)) \lvert \nabla u \rvert}_{\textrm{Shock}},
$$
in which $g: [0, \infty) \to (0, 1]$ is a decreasing function with $g(0) = 1$, $G_{\alpha}$ is a Gaussian with standard deviation $\alpha$, and $\vec{w}$ is the dominant eigenvector of the structure tensor. It is clear then that $g$ switches between applying diffusion and shock: if the gradient of the image is small, we mostly apply diffusion, but if the gradient is large, we mostly apply shock. This makes sense, since a large gradient implies that there is a feature there, which we would like to sharpen up. 

The signum in the shock term switches between erosion and dilation. If the second derivative with respect to the dominant eigenvector of the structure tensor is positive, then we perform erosion (defined by the PDE $\partial_t u = -\lvert \nabla u \rvert$); otherwise we perform dilation (defined by the PDE $\partial_t u = \lvert \nabla u \rvert$). In regularised DS, the signum is replaced with a soft signum, so that the selection of erosion vs dilation is less sensitive to noise.

The signum of the second derivative of the dominant eigenvector of the structure tensor is not unlike the convexity criterion we know from studying vesselness; perhaps we could replace it?

What is the correct way to extend DS to $SE(2)$? It would make sense to keep the gradients and Laplacian. For the selection of erosion vs dilation we could again look at the vesselness convexity criterion. For switching between diffusion and shock, we could maybe use some sort of line/edge detector.

In [ ]:
import taichi as ti
ti.init(arch=ti.gpu, debug=False, device_memory_GB=3.5) #, kernel_profiler=True) # Use less than 4 so that we don't mix RAM and VRAM (?)
import numpy as np
import scipy as sp
# from PIL import Image
from datetime import datetime
import matplotlib.pyplot as plt
# %matplotlib widget
import dsfilter

In [2]:
date = datetime.today().strftime("%y-%m-%d")
log_folder = "..\\..\\Experiments For Paper\\Inpainting\\Straight Lines"
savefigures = False

In [3]:
# "black_on_white" "black_and_white_on_grey" "diagonal" "cross"
test_case = "black_on_white"

In [4]:
match test_case:
    case "black_on_white": # Grid of lines (black)
        dim_I, dim_J = 256, 256
        u_ground_truth = np.ones((dim_I, dim_J)) * 255.
        N_lines = 4
        offset = dim_I // (N_lines + 1)
        for k in range(N_lines):
            centre = (k + 1) * offset
            u_ground_truth[:, (centre-2):(centre+3)] = 0.
            u_ground_truth[(centre-2):(centre+3), :] = 0.

        xs, ys = np.meshgrid(np.linspace(-1, 1, dim_I), np.linspace(-1, 1, dim_J))
        l = 0.4
        mask = (xs**2 < l) * (ys**2 < l)

        u = u_ground_truth.copy()
        u[mask] = 255.

        dim_K = 16
        # ξ = 0.2
        ξ = 0.02

        G_D_inv = 1.8 * np.array((1., 0.1, 0.))
        G_S_inv = np.array((1., 1., 0.))
        # Internal regularisation for switching between dilation and erosion.
        σ_1, σ_2, σ_3 = np.array((2.5, 2.5, 0.6))
        # External regularisation for switching between dilation and erosion.
        ρ_1, ρ_2, ρ_3 = np.array((1., 1., 0.6))
        # Internal and external regularisation of gradient for switching between diffusion and shock.
        ν_1, ν_2, ν_3 = np.array((2.5, 2.5, 0.6))
        λ = 0.02 # Contrast parameter for switching between diffusion and shock.
        ε = 0.5 # Regularisation parameter for signum.

        # # G_D_inv = 1.8 * np.array((ξ**-2, ξ**-2, 1.))
        # # G_S_inv = np.array((ξ**-2, ξ**-2, 1.))
        # G_D_inv = 1.8 * np.array((ξ**-2, ξ**-2, 0.))
        # G_S_inv = np.array((ξ**-2, ξ**-2, 0.))
        # # Internal regularisation for switching between dilation and erosion.
        # σ = 2.5
        # # External regularisation for switching between dilation and erosion.
        # ρ = 1.
        # # Internal regularisation of gradient for switching between diffusion and shock.
        # ν = 2.5
        # λ = 0.2 * 0.05 / ξ # Contrast parameter for switching between diffusion and shock.
        # ε = 0.5 / ξ**2 # Regularisation parameter for signum.

        T_short = 0.2
        T_medium = 2.
        T_long = 10.
        T_mega_long = 250.

        # T_short = 0.2 * ξ**2
        # T_medium = 2. * ξ**2
        # T_long = 10. * ξ**2
        # T_mega_long = 150. * ξ**2

        σ_R2 = 2.5
        ρ_R2 = 1.6 * σ_R2
        ν_R2 = 1.6 * σ_R2
        λ_R2 = 1.
        ε_R2 = 0.15 * λ_R2

        T_short_R2 = 0.4
        T_medium_R2 = 2.
        T_long_R2 = 100.
        T_mega_long_R2 = 350.

    case "black_and_white_on_grey": # Grid of lines (alternating black and white)
        dim_I, dim_J = 256, 256
        u_ground_truth = np.ones((dim_I, dim_J)) * 0.5 * 255.
        N_lines = 4 # 7
        offset = dim_I // (N_lines + 1)
        colour = 0. # black
        for k in range(N_lines):
            centre = (k + 1) * offset
            u_ground_truth[:, (centre-2):(centre+3)] = 255. - colour
            u_ground_truth[(centre-2):(centre+3), :] = colour
            colour = 255. - colour
        u_ground_truth = sp.ndimage.gaussian_filter(u_ground_truth, 1)

        xs, ys = np.meshgrid(np.linspace(-1, 1, dim_I), np.linspace(-1, 1, dim_J))
        l = 0.4
        mask = (xs**2 < l) * (ys**2 < l)

        u = u_ground_truth.copy()
        u[mask] = 0.5 * 255.
        
        dim_K = 16
        ξ = 8 / dim_K
        G_D_inv = 1.8 * np.array((1., 0.1, 0.0))
        G_S_inv = np.array((1., 1., 0.0))
        # G_D_inv = 1.8 * np.array((1., 0.1, 0.1 * (ξ/16.)**2))
        # G_S_inv = np.array((1., 1., (ξ/16.)**2))
        # Internal regularisation for switching between dilation and erosion.
        σ_1, σ_2, σ_3 = np.array((2.5, 2.5, 0.6))
        # External regularisation for switching between dilation and erosion.
        ρ_1, ρ_2, ρ_3 = np.array((1., 1., 0.6))
        # Internal and external regularisation of gradient for switching between diffusion and shock.
        ν_1, ν_2, ν_3 = np.array((2.5, 2.5, 0.6))
        λ = 0.01 # Contrast parameter for switching between diffusion and shock.
        ε = 0.5 # Regularisation parameter for signum.

        # # G_D_inv = 1.8 * np.array((ξ**-2, 0.1 * ξ**-2, 0.))
        # # G_S_inv = np.array((ξ**-2, ξ**-2, 0.))
        # # Internal regularisation for switching between dilation and erosion.
        # σ = 2.5
        # # External regularisation for switching between dilation and erosion.
        # ρ = 1.
        # # Internal regularisation of gradient for switching between diffusion and shock.
        # ν = 2.5
        # λ = 0.01 # Contrast parameter for switching between diffusion and shock.
        # ε = 0.5 # Regularisation parameter for signum.

        T_short = 0.2
        T_medium = 2.
        # T_long = 10.
        T_long = 100.
        T_mega_long = 350.


        σ_R2 = 2.5
        ρ_R2 = 1.6 * σ_R2
        ν_R2 = 1.6 * σ_R2
        λ_R2 = 1.
        ε_R2 = 0.15 * λ_R2

        T_short_R2 = 0.4
        T_medium_R2 = 2.
        T_long_R2 = 100.
        T_mega_long_R2 = 350.

        # T_short = 0.07 * ξ**2
        # T_medium = 1. * ξ**2
        # T_long = 5. * ξ**2
        # T_mega_long = 100. * ξ**2

    case "diagonal": # Lines with diagonal (black)
        dim_I, dim_J = 256, 256
        u_ground_truth = np.ones((dim_I, dim_J)) * 255.
        N_lines = 4
        offset = dim_I // (N_lines + 1)
        for k in range(N_lines):
            centre = (k + 1) * offset
            u_ground_truth[:, (centre-2):(centre+3)] = 0.

        xs, ys = np.meshgrid(np.linspace(-1, 1, dim_I), np.linspace(-1, 1, dim_J))
        diagonal = (np.abs(xs - ys) < 0.03)
        u_ground_truth[diagonal] = 0.
        l = 0.4
        mask = (xs**2 < l) * (ys**2 < l)

        u = u_ground_truth.copy()
        u[mask] = 255.

        dim_K = 32
        ξ = 8 / dim_K
        G_D_inv = 1.8 * np.array((1., 0.1, 0.0))
        G_S_inv = np.array((1., 1., 0.0))
        # # Internal regularisation for switching between dilation and erosion.
        # σ_1, σ_2, σ_3 = np.array((2.5, 2.5, 0.6))
        # # External regularisation for switching between dilation and erosion.
        # ρ_1, ρ_2, ρ_3 = np.array((1., 1., 0.6))
        # # Internal and external regularisation of gradient for switching between diffusion and shock.
        # ν_1, ν_2, ν_3 = np.array((2.5, 2.5, 0.6))
        # λ = 0.02 # Contrast parameter for switching between diffusion and shock.
        # ε = 0.5 # Regularisation parameter for signum.

        # G_D_inv = 1.8 * np.array((ξ**-2, 0.1 * ξ**-2, 0.))
        # G_S_inv = np.array((ξ**-2, ξ**-2, 0.))
        # Internal regularisation for switching between dilation and erosion.
        σ = 2.5
        # External regularisation for switching between dilation and erosion.
        ρ = 1.
        # Internal regularisation of gradient for switching between diffusion and shock.
        ν = 2.5
        λ = 0.02 # Contrast parameter for switching between diffusion and shock.
        ε = 0.5 # Regularisation parameter for signum.

        T_short = 0.2
        T_medium = 2.
        T_long = 10.
        T_mega_long = 250.

    case "cross": # Cross
        dim_I, dim_J = 256, 256
        xs, ys = np.meshgrid(np.linspace(-1, 1, dim_I), np.linspace(-1, 1, dim_J))
        vertical = (-0.2 < xs) * (xs < 0.2)
        horizontal = (-0.2 < ys) * (ys < 0.2)
        u_ground_truth = np.zeros((dim_I, dim_J))
        u_ground_truth[vertical + horizontal] = 1. * 255.

        mask = (-0.3 < xs) * (xs < 0.3) * (-0.3 < ys) * (ys < 0.3)
        u = u_ground_truth.copy()
        u[mask] = 0.

        dim_K = 16
        ξ = 8 / dim_K
        G_D_inv = 1.8 * np.array((1., 1., 0.0))
        G_S_inv = np.array((1., 1., 0.0))
        # # Internal regularisation for switching between dilation and erosion.
        # σ_1, σ_2, σ_3 = np.array((5., 5., 0.8))
        # # External regularisation for switching between dilation and erosion.
        # ρ_1, ρ_2, ρ_3 = np.array((2.5, 2.5, 0.8))
        # # Internal and external regularisation of gradient for switching between diffusion and shock.
        # ν_1, ν_2, ν_3 = np.array((5., 5., 0.8))
        # λ = 0.005 # Contrast parameter for switching between diffusion and shock.
        # ε = 0.5 # Regularisation parameter for signum.

        # G_D_inv = 1.8 * np.array((ξ**-2, 0.1 * ξ**-2, 0.))
        # G_S_inv = np.array((ξ**-2, ξ**-2, 0.))
        # Internal regularisation for switching between dilation and erosion.
        σ = 5.
        # External regularisation for switching between dilation and erosion.
        ρ = 2.5
        # Internal regularisation of gradient for switching between diffusion and shock.
        ν = 5.
        λ = 0.005 # Contrast parameter for switching between diffusion and shock.
        ε = 0.5 # Regularisation parameter for signum.

        T_short = 0.2
        T_medium = 2.
        T_long = 10.
        T_mega_long = 250.

u = sp.ndimage.gaussian_filter(u, 1.) # Smooth for well-posed lifting.

clip = (u.min(), u.max())

mask_orig = 1 - mask.astype(int)
mask = sp.ndimage.binary_erosion(mask_orig, iterations=10, border_value=1).astype(int) # Deal with boundary artefacts.

Is, Js, Ks = np.indices((dim_I, dim_J, dim_K))
x_min, x_max = 0., dim_I - 1.
y_min, y_max = 0., dim_J - 1.
θ_min, θ_max = 0., 2 * np.pi
dxy = (x_max - x_min) / (dim_I - 1)
dθ = (θ_max - θ_min) / dim_K
xs, ys, θs = dsfilter.SE2.utils.coordinate_array_to_real(Is, Js, Ks, x_min, y_min, θ_min, dxy, dθ)

In [ ]:
fig, ax, cbar = dsfilter.visualisations.plot_image_array(u, x_min, x_max, y_min, y_max, cmap="gray", figsize=(6, 5))
ax.set_title("$u$")
fig.colorbar(cbar, ax=ax);

In [6]:
if savefigures:
    fig, ax, _ = dsfilter.visualisations.plot_image_array(u, x_min, x_max, y_min, y_max, cmap="gray", figsize=(6, 5))
    ax.set_axis_off()
    fig.savefig(f"{log_folder}\\test_{test_case}.png", bbox_inches="tight", pad_inches=0.)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
_, _, cbar = dsfilter.visualisations.plot_image_array(u, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title("$u$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(mask, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title("Mask")
fig.colorbar(cbar, ax=ax[1]);

### Orientation Score

In [8]:
cws_check = dsfilter.orientationscore.cakewavelet_stack(dim_I, dim_K, Gaussian_σ=dim_I/8)

In [ ]:
K = 1
print(θs[0, 0, K])
fig, ax, cbar = dsfilter.visualisations.plot_image_array(cws_check.real[K], x_min, x_max, y_min, y_max, cmap="gray")
ax.set_title("$\psi$")
fig.colorbar(cbar, ax=ax);

In [ ]:
cws = dsfilter.orientationscore.cakewavelet_stack(dim_I, dim_K, Gaussian_σ=dim_I/8)
U = dsfilter.orientationscore.wavelet_transform(u, cws).real
U = np.transpose(U, axes=(1, 2, 0)) # x, y, θ
Mask = np.transpose(np.array([mask for _ in range(dim_K)]), axes=(1, 2, 0)) # x, y, θ
U.shape, Mask.shape

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(1, 3, figsize=(18, 5))
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K] * mask, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title(f"{K} $U$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K + 2] * mask, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title(f"{K + 2} $U$")
fig.colorbar(cbar, ax=ax[1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K + 4] * mask, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2])
ax[2].set_title(f"{K + 4} $U$")
fig.colorbar(cbar, ax=ax[2]);

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18, 5))
_, _, cbar = dsfilter.visualisations.plot_image_array(u, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title("$u$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title("Reconstruction")
fig.colorbar(cbar, ax=ax[1])
_, _, cbar = dsfilter.visualisations.plot_image_array(u - U.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2])
ax[2].set_title("Reconstruction error")
fig.colorbar(cbar, ax=ax[2]);

### Preprocess Orientation Score
To remove artefacts in the orientation score caused by lifting masked data, we set the data within the (dilated) mask to a constant.

In [13]:
U_preprocessed = dsfilter.SE2.utils.clean_mask_boundaries(U, Mask)

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(3, 3, figsize=(18, 15))
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"{K} $U$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"{K + 2} $U$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 2])
ax[0, 2].set_title(f"{K + 4} $U$")
fig.colorbar(cbar, ax=ax[0, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_preprocessed[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"{K} preprocessed $U$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_preprocessed[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"{K + 2} preprocessed $U$")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_preprocessed[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 2])
ax[1, 2].set_title(f"{K + 4} preprocessed $U$")
fig.colorbar(cbar, ax=ax[1, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K] * mask, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2, 0])
ax[2, 0].set_title(f"{K} preprocessed $U$")
fig.colorbar(cbar, ax=ax[2, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K + 2] * mask, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2, 1])
ax[2, 1].set_title(f"{K + 2} preprocessed $U$")
fig.colorbar(cbar, ax=ax[2, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K + 4] * mask, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2, 2])
ax[2, 2].set_title(f"{K + 4} preprocessed $U$")
fig.colorbar(cbar, ax=ax[2, 2]);

### $\mathbb{R}^2$ DS Inpainting

In [ ]:
u_filtered_short = dsfilter.DS_inpainting_R2(u, mask, T_short_R2, σ_R2, ρ_R2, ν_R2, λ_R2, ε=ε_R2, dxy=dxy)
u_filtered_medium = dsfilter.DS_inpainting_R2(u, mask, T_medium_R2, σ_R2, ρ_R2, ν_R2, λ_R2, ε=ε_R2, dxy=dxy)
u_filtered_long = dsfilter.DS_inpainting_R2(u, mask, T_long_R2, σ_R2, ρ_R2, ν_R2, λ_R2, ε=ε_R2, dxy=dxy)
u_filtered_mega_long = dsfilter.DS_inpainting_R2(u, mask, 10 * T_mega_long_R2, σ_R2, ρ_R2, ν_R2, λ_R2, ε=ε_R2, dxy=dxy)

In [ ]:
fig, ax, cbar = dsfilter.visualisations.plot_image_array((u_filtered_long - u_filtered_mega_long), x_min, x_max, y_min, y_max, cmap="gray")
fig.colorbar(cbar, ax=ax);

In [ ]:
fig, ax, cbar = dsfilter.visualisations.plot_image_array(u_filtered_mega_long, x_min, x_max, y_min, y_max, clip=clip, cmap="gray", figsize=(6, 5))
ax.set_title("$\int_\\theta U(\\theta) d\\theta$")
fig.colorbar(cbar, ax=ax);

In [18]:
if savefigures:
    fig, ax, _ = dsfilter.visualisations.plot_image_array(u_filtered_mega_long, x_min, x_max, y_min, y_max, cmap="gray", figsize=(6, 5))
    ax.set_axis_off()
    fig.savefig(f"{log_folder}\\test_{test_case}_result_R2.png", bbox_inches="tight", pad_inches=0.)

### Run DS Inpainting

In [ ]:
# u_filtered_short = dsfilter.DS_inpainting_LI(U_preprocessed, Mask, θs, ξ, T_short, G_D_inv, G_S_inv, σ, ρ, ν, λ, ε=ε, dxy=dxy)
# u_filtered_medium = dsfilter.DS_inpainting_LI(U_preprocessed, Mask, θs, ξ, T_medium, G_D_inv, G_S_inv, σ, ρ, ν, λ, ε=ε, dxy=dxy)
# u_filtered_long = dsfilter.DS_inpainting_LI(U_preprocessed, Mask, θs, ξ, T_long, G_D_inv, G_S_inv, σ, ρ, ν, λ, ε=ε, dxy=dxy)
# u_filtered_mega_long = dsfilter.DS_inpainting_LI(U_preprocessed, Mask, θs, ξ, 50 * T_mega_long, G_D_inv, G_S_inv, σ, ρ, ν, λ, ε=ε, dxy=dxy)
u_filtered_short = dsfilter.DS_inpainting_spatial(U_preprocessed, Mask, θs, T_short, G_D_inv, G_S_inv, σ_1, σ_3, ρ_1, ρ_3, ν_1, ν_3, λ, ε=ε, dxy=dxy)
u_filtered_medium = dsfilter.DS_inpainting_spatial(U_preprocessed, Mask, θs, T_medium, G_D_inv, G_S_inv, σ_1, σ_3, ρ_1, ρ_3, ν_1, ν_3, λ, ε=ε, dxy=dxy)
u_filtered_long = dsfilter.DS_inpainting_spatial(U_preprocessed, Mask, θs, T_long, G_D_inv, G_S_inv, σ_1, σ_3, ρ_1, ρ_3, ν_1, ν_3, λ, ε=ε, dxy=dxy)
u_filtered_mega_long = dsfilter.DS_inpainting_spatial(U_preprocessed, Mask, θs, T_mega_long, G_D_inv, G_S_inv, σ_1, σ_3, ρ_1, ρ_3, ν_1, ν_3, λ, ε=ε, dxy=dxy)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_short.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", clip=clip, fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$T = {T_short:.2f}$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_medium.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", clip=clip, fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$T = {T_medium:.2f}$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_long.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", clip=clip, fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$T = {T_long:.2f}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_mega_long.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", clip=clip, fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$T = {T_mega_long:.2f}$")
fig.colorbar(cbar, ax=ax[1, 1]);

In [ ]:
K = 0
fig, ax = plt.subplots(1, 5, figsize=(30, 5))
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_mega_long[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title(f"$\\theta = {θs[0, 0, K]:.2f}$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_mega_long[..., K + dim_K//16], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title(f"$\\theta = {θs[0, 0, K + dim_K//16]:.2f}$")
fig.colorbar(cbar, ax=ax[1])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_mega_long[..., K + 2*dim_K//16], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2])
ax[2].set_title(f"$\\theta = {θs[0, 0, K + 2*dim_K//16]:.2f}$")
fig.colorbar(cbar, ax=ax[2])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_mega_long[..., K + 4*dim_K//16], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[3])
ax[3].set_title(f"$\\theta = {θs[0, 0, K + 4*dim_K//16]:.2f}$")
fig.colorbar(cbar, ax=ax[3])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_mega_long.sum(-1), x_min, x_max, y_min, y_max, clip=clip, cmap="gray", fig=fig, ax=ax[4])
ax[4].set_title("$\int_\\theta U(\\theta) d\\theta$")
fig.colorbar(cbar, ax=ax[4]);
if savefigures:
    fig.savefig(f"{log_folder}\\{test_case}_{date}.svg", bbox_inches="tight", dpi=100)

In [ ]:
fig, ax, cbar = dsfilter.visualisations.plot_image_array(u_filtered_mega_long.sum(-1), x_min, x_max, y_min, y_max, clip=clip, cmap="gray", figsize=(12, 10))
ax.set_title("$\int_\\theta U(\\theta) d\\theta$")
fig.colorbar(cbar, ax=ax);

In [23]:
if savefigures:
    fig, ax, _ = dsfilter.visualisations.plot_image_array(u_filtered_mega_long.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", figsize=(6, 5), clip=clip)
    ax.set_axis_off()
    fig.savefig(f"{log_folder}\\test_{test_case}_result_SE(2).png", bbox_inches="tight", pad_inches=0.)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(np.clip(u_filtered_short.sum(-1), 0., 255.) - u_ground_truth, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$T = {T_short:.2f}$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(np.clip(u_filtered_medium.sum(-1), 0., 255.) - u_ground_truth, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$T = {T_medium:.2f}$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(np.clip(u_filtered_long.sum(-1), 0., 255.) - u_ground_truth, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$T = {T_long:.2f}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(np.clip(u_filtered_mega_long.sum(-1), 0., 255.) - u_ground_truth, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$T = {T_mega_long:.2f}$")
fig.colorbar(cbar, ax=ax[1, 1]);